# **Proyecto 1 - Etapa I**

Integrantes:
*   Kevin Gamez
*   Andres Bravo
*   Sara Plazas

# Entendimiento de los datos

In [1]:
#Instalaciones

#Libreria para remplazar numeros por palabras
!pip install num2words
#Instalar  pandas profiler
!pip install [https://github.com/pandas-profiling/pandas-profiling/archive/master.zip](https://github.com/pandas-profiling/pandas-profiling/archive/master.zip)

#deteccion de lenguaje para eliminar entradas que no esten en español
!pip install langdetect
#Procesamientno de lenguaje natural en espaañol
!pip install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 KB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=aa8d8cead6d5c26442c10675590d892df8e4bdca153561c8b3cf85b5ae20460e
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt
/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `pip install [https://github.com/pandas-profiling/pandas-profiling/archive/master.zip](https://github.com/pandas-profiling/pandas-profiling/archive/master.zip)'
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langd

In [2]:
#imports para procesamiento de texto

#tokenizacion y lematizacion
import stanza
#Para integrar pasos de la limpieza adicionales
from stanza.pipeline.processor import Processor, register_processor
#paquete español 
stanza.download('es') 

#Para manejo de numeros, singluares, plurarles en lenguaje
from num2words import num2words
#Deteccion de lenguaje
from langdetect import detect

import nltk
# para identificar palabras comunes a todos los textos 
nltk.download('stopwords')
from nltk.corpus import stopwords

#Operaciones con expresiones regulares
import re
#Unicode character database
import unicodedata

INFO:stanza:Downloading default packages for language: es (Spanish) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
#Imports generales para analisis de datos y ML
import pandas as pd
import numpy as np
import sys
import statistics

from pandas_profiling import ProfileReport

import sklearn
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

# Entendimiento de los datos

In [6]:
# Ruta en la que se encuentra la base de datos de peliculas
path = "MovieReviews.csv"
df = pd.read_csv(path, sep = ",", encoding='utf-8')

df.head()

,Unnamed: 0,review_es,sentimiento
0,0,Si está buscando una película de guerra típica...,positivo
1,1,Supongo que algunos directores de películas de...,positivo
2,2,Es difícil contarle más sobre esta película si...,positivo
3,3,"La película comienza muy lentamente, con el es...",positivo
4,4,Esta película es verdadera acción en su máxima...,positivo


In [7]:
#Eliminamos la columna Unnamed:0 que no esta en el diccionario
df = df.drop('Unnamed: 0', axis = 1)
df.shape

(5000, 2)

In [8]:
df.dtypes

review_es      object
sentimiento    object
dtype: object

#Imprimimos reporte de pandas Profiler

In [9]:
textos = df.copy()
#Longitud del comentario (numero de caracteres)
textos['Conteo'] = [len(x) for x in textos['review_es']]
#Palabra(s) mas frecuente(s)
textos['Moda'] = [max(set(x.split(' ')), key = x.split(' ').count) for x in df['review_es']]
#Longitud de la palabra mas larga del comentario
textos['Max'] = [[max([len(x) for x in i.split(' ')])][0] for i in textos['review_es']]
#Longitud de la palabra mas corta del comentario
textos['Min'] = [[min([len(x) for x in i.split(' ')])][0] for i in textos['review_es']]

ProfileReport(textos)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
#Conteo de entradas nulas (esta informacion esta en el reporte pero decidimos ponerla aparte)
df.isnull().sum()

review_es      0
sentimiento    0
dtype: int64

In [11]:
#verificamos entradas en otros idiomas
df[df.review_es.map(detect)  != 'es']

,review_es,sentimiento
34,"Even a decade after ""Frontline"" aired on the A...",positivo
62,This film is so different to anything you woul...,positivo
100,"First of all, ignore the comment about how Sou...",positivo
149,I never expected such an old film to be as imp...,positivo
162,This is a really great short from Hal Roach Th...,positivo
...,...,...
4899,I have this film out of the library right now ...,negativo
4952,"Navy Seals is an ignorant, racist and complace...",negativo
4983,Matt Cvetic is a loyal communist in a Pittsbur...,negativo
4988,The good news for IMDb is that this movie was ...,negativo


# Preparacion de los datos

In [12]:
#Eliminamos Duplicados
df = df.drop_duplicates()
#Eliminamos entradas en otros idiomas
df = df.drop(df[df.review_es.map(detect)  != 'es'].index)

In [13]:
 #verificamos cuantos datos quedan
 df.shape

(4801, 2)

## Limpieza, tokenizacion y lemmatizacion

In [14]:
#Remplaza los numeros por su representacion en palabras
def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = num2words(word, lang='es')
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words
#Remueve todo caracter no latino (conserva espacios y numeros)
def remove_nonlatin(words):
  new_words = []
  for word in words:
    new_word = ''
    for ch in word:
      if unicodedata.name(ch).startswith(('LATIN', 'DIGIT', 'SPACE')):
        new_word += ch
    new_words.append(new_word)
  return new_words

#Remueve palabras comunes que no aportan informacion
def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    s = set(stopwords.words('spanish'))
    for word in words:
        if word not in s:
            new_words.append(word)
    return new_words

#Remueve puntuacion
def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = ''
    for word in words:
            new_words += re.sub(r'[^\w\s]', ' ', word)
    return new_words

 #Procesamiento de cada review usando stanza
def tokenLemma():
  df['words'] = df['review_es'].apply(remove_punctuation)
  #Creamos un pipeline para tokenizacion y lematizacion
  nlp = stanza.Pipeline('es', processors = 'tokenize,mwt,pos,lemma', use_gpu=True)
  in_docs = [stanza.Document([], text=d) for d in df.words]
  return nlp(in_docs)

#Funcion secundaria para procesar cada token
def procesamientoPalabras(words):
    words = remove_nonlatin(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    return words

#Funcion principal para el pre-procesamiento
def preprocessing():
  out_docs = tokenLemma()
  palabras = []

  for doc in out_docs:
    reviewAct = []
    for sentence in doc.sentences:
      for word in sentence.words:
        if(word.pos != 'PUNCT' and word.pos != 'SYM'):
          reviewAct.append(word.lemma.lower())
    palabras.append(reviewAct)

  df['words'] = palabras
  df['words'] = df['words'].apply(procesamientoPalabras)

In [15]:
#Comente la siguiente linea si desea cargar el archivo previamente limpiado
#preprocessing()

#Descomentar estas lineas para cargar el archivo Limpio
df = pd.read_csv("df5000nuevo.csv", sep = ";", encoding='utf-8')
import ast
df['words'] = df['words'].apply(ast.literal_eval)

#Imprimimos resultado
df.head()

,review_es,sentimiento,words
0,Si está buscando una película de guerra típica...,positivo,"[si, buscar, película, guerra, típico, ser, as..."
1,Supongo que algunos directores de películas de...,positivo,"[suponer, alguno, director, película, lujo, se..."
2,Es difícil contarle más sobre esta película si...,positivo,"[ser, difícil, contar, película, estropear, di..."
3,"La película comienza muy lentamente, con el es...",positivo,"[película, comenzar, lentamente, estilo, vida,..."
4,Esta película es verdadera acción en su máxima...,positivo,"[película, ser, verdadero, acción, máximo, exp..."


In [16]:
#Unimos los tokens para volver a obtener un string por cada reseña
df['words'] = df['words'].apply(lambda x: ' '.join(map(str, x)))
df

,review_es,sentimiento,words
0,Si está buscando una película de guerra típica...,positivo,si buscar película guerra típico ser así nota ...
1,Supongo que algunos directores de películas de...,positivo,suponer alguno director película lujo sentado ...
2,Es difícil contarle más sobre esta película si...,positivo,ser difícil contar película estropear disfruta...
3,"La película comienza muy lentamente, con el es...",positivo,película comenzar lentamente estilo vida walla...
4,Esta película es verdadera acción en su máxima...,positivo,película ser verdadero acción máximo expresión...
...,...,...,...
4796,Las bonitas visuales y muchas peleas no hacen ...,negativo,bonito visual pelea hacer buen película ser pr...
4797,"""Criminal decente ordinario"" es triste porque ...",negativo,criminal decente ordinario ser triste obviamen...
4798,Savage Island (2003) es una película coja.Es m...,negativo,savage island dos mil tres ser película cojo s...
4799,Quien escribió el script para esta película no...,negativo,escribir script película merecer trabajar holl...


### Volvemos a imprimir el reporte para verificar que el procesamiento fue correcto

In [17]:
textos = df.copy()
#Longitud del comentario (numero de caracteres)
textos['Conteo'] = [len(x) for x in textos['words']]
#Palabra(s) mas frecuente(s)
textos['Moda'] = [max(set(x.split(' ')), key = x.split(' ').count) for x in df['words']]
#Longitud de la palabra mas larga del comentario
textos['Max'] = [[max([len(x) for x in i.split(' ')])][0] for i in textos['words']]
#Longitu de la palabra mas corta del comentario
textos['Min'] = [[min([len(x) for x in i.split(' ')])][0] for i in textos['words']]

ProfileReport(textos)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

## Construimos entrenamiento-test y creamos representación Tfidf

In [18]:
df = df.drop('review_es', axis = 1)
entrenamiento, test = sklearn.model_selection.train_test_split(df, test_size=0.2, random_state=0)

X_train = entrenamiento['words']
X_test = test['words']
y_train = entrenamiento['sentimiento']
y_test = test['sentimiento']

In [19]:
vectorizer = TfidfVectorizer()
train_vectors = vectorizer.fit_transform(X_train)
test_vectors = vectorizer.transform(X_test)
print(train_vectors.shape, test_vectors.shape)

(3840, 30605) (961, 30605)


# Modelado y evaluacion

## Naive Bayes


In [20]:
#Para realizar validación cruzada particionamos el conjunto de entrenamiento
# Fijemos el número de particiones. Utilizaremos K = 5
particiones = KFold(n_splits=5, shuffle=True, random_state = 0)

# Establecemos el espacio de búsqueda para los hiperparámetros en especial los que nos permiten controlar la complejidad y el rendimiento del modelo 
param_grid = {'alpha':[0.2, 0.4, 0.6, 0.8, 1],'fit_prior':[True, False]}

#Creamos la instacia del modelo a entrenar
bayes = MultinomialNB()

# Utilizamos GridSearch sobre el grid definido y con 4 particiones en la validación cruzada.
mejor_modelo = GridSearchCV(bayes, param_grid, cv=particiones, refit = True, scoring = 'accuracy')
# Ajuste del modelo
mejor_modelo.fit(train_vectors, y_train)

#Vemos que parametros escogio
mejor_modelo.best_params_

{'alpha': 1, 'fit_prior': False}

In [21]:
clf = mejor_modelo.best_estimator_
predicted = clf.predict(test_vectors)
class_report = classification_report(y_test, predicted)
print("\nClassification Report")
print(class_report)


Classification Report
              precision    recall  f1-score   support

    negativo       0.83      0.88      0.85       491
    positivo       0.87      0.80      0.84       470

    accuracy                           0.84       961
   macro avg       0.85      0.84      0.84       961
weighted avg       0.85      0.84      0.84       961



In [22]:
accuracy_score(y_test, predicted)

0.8449531737773153

In [23]:
test['pred'] = predicted
test

,sentimiento,words,pred
2255,positivo,película basar arte frank frazetta mítico ilus...,positivo
608,positivo,ser rehash película reciente solo tener menos ...,positivo
2856,negativo,película niño haber ver largo año ser probable...,negativo
1889,positivo,realmente envejecer medio ver película cuarent...,positivo
1519,positivo,ser ver película libro firmar lexington ky ano...,positivo
...,...,...,...
4595,negativo,ser difícil esperar realmente ser peor películ...,negativo
891,positivo,lana turner demostrar realmente poder bailar t...,positivo
1005,positivo,jodie foster cherie currie antiguo cantante pr...,positivo
1940,positivo,primero foto hablar haber regresar olvido ahor...,positivo


## Logistic Regression

In [24]:
from matplotlib import style
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

In [25]:
entrenamiento, test = sklearn.model_selection.train_test_split(df, test_size=0.2, random_state=0)

X_train = entrenamiento['words']
X_test = test['words']
y_train = entrenamiento['sentimiento']
y_test = test['sentimiento']

vectorizer = TfidfVectorizer()
train_vectors = vectorizer.fit_transform(X_train)
test_vectors = vectorizer.transform(X_test)

In [26]:
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')
# parameter grid
parameters = {
    'penalty' : ['l1','l2', 'elasticnet', None], 
    'C'       : np.logspace(-10,10,3),
    'solver'  : ['newton-cg', 'lbfgs', 'liblinear'],
}

In [27]:

from sklearn.model_selection import RepeatedKFold
cv = RepeatedKFold(n_splits=20, n_repeats= 10, random_state=0)

logreg = LogisticRegression()
clf = GridSearchCV(logreg,                    # model
                   param_grid = parameters,   # hyperparameters
                   scoring='accuracy',  
                    cv=cv,      # metric for scoring
                   n_jobs=-1)  

In [28]:
clf.fit(train_vectors,y_train)

GridSearchCV(cv=RepeatedKFold(n_repeats=10, n_splits=20, random_state=0),
             estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': array([1.e-10, 1.e+00, 1.e+10]),
                         'penalty': ['l1', 'l2', 'elasticnet', None],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear']},
             scoring='accuracy')

In [29]:
print("Tuned Hyperparameters :", clf.best_params_)
print("Accuracy :",clf.best_score_)
model = clf.best_estimator_

Tuned Hyperparameters : {'C': 10000000000.0, 'penalty': 'l2', 'solver': 'lbfgs'}
Accuracy : 0.849140625


In [30]:
predictions = model.predict(test_vectors)

#Medimos exactitud
score = model.score(test_vectors, y_test)

print("Exactitud predicción test: " + str(score))

Exactitud predicción test: 0.8574401664932362


In [31]:
from sklearn import metrics

cm = metrics.confusion_matrix(y_test, predictions)
print(cm)


[[418  73]
 [ 64 406]]


In [32]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
class_report = classification_report(y_test, predictions)
print("\nClassification Report")
print(class_report)


Classification Report
              precision    recall  f1-score   support

    negativo       0.87      0.85      0.86       491
    positivo       0.85      0.86      0.86       470

    accuracy                           0.86       961
   macro avg       0.86      0.86      0.86       961
weighted avg       0.86      0.86      0.86       961



## Random Forest

In [33]:
#Modelo con parametros por defectto para comparar resultados
clf = RandomForestClassifier(random_state=0)
clf.fit(train_vectors, y_train)
ytest_pred = clf.predict(test_vectors)

clf = RandomForestClassifier(random_state=0)
clf.fit(train_vectors, y_train)
predicted = clf.predict(test_vectors)
class_report = classification_report(y_test, predicted)
print("\nClassification Report")
print(class_report)


Classification Report
              precision    recall  f1-score   support

    negativo       0.81      0.84      0.83       491
    positivo       0.83      0.80      0.81       470

    accuracy                           0.82       961
   macro avg       0.82      0.82      0.82       961
weighted avg       0.82      0.82      0.82       961



In [34]:
#Para realizar validación cruzada particionamos el conjunto de entrenamiento
# Fijemos el número de particiones.
particiones = KFold(n_splits=4, shuffle=True, random_state = 0)

# Establecemos el espacio de búsqueda para los hiperparámetros en especial los que nos permiten controlar la complejidad y el rendimiento del modelo 
param_grid = {'criterion':['gini', 'entropy', 'log_loss'],'min_samples_split':[2, 4, 6],'max_features':[10, 100, 1000]}

#Creamos la instacia del modelo a entrenar
forest = RandomForestClassifier(random_state=0)

# Utilizamos GridSearch sobre el grid definido y con 4 particiones en la validación cruzada.
mejor_modelo = GridSearchCV(forest, param_grid, cv=particiones, refit = True, scoring = 'accuracy')
# Ajuste del modelo
mejor_modelo.fit(train_vectors, y_train)

#Vemos que parametros escogio
print(mejor_modelo.best_params_)

clf = mejor_modelo.best_estimator_
predicted = clf.predict(test_vectors)
class_report = classification_report(y_test, predicted)
print("\nClassification Report")
print(class_report)

{'criterion': 'entropy', 'max_features': 100, 'min_samples_split': 4}

Classification Report
              precision    recall  f1-score   support

    negativo       0.84      0.83      0.84       491
    positivo       0.82      0.84      0.83       470

    accuracy                           0.83       961
   macro avg       0.83      0.83      0.83       961
weighted avg       0.83      0.83      0.83       961



In [35]:
accuracy_score(y_test, predicted)

0.8335067637877211